In [1]:
import pandas as pd
import pickle
import ete3
root = '/home/tobiassonva/data/eukgen/'
%cd {root}
!pwd

/vf/users/tobiassonva/data/eukgen
/vf/users/tobiassonva/data/eukgen


In [2]:
#reformat header by keeping only accession. Save header in separate tsv. Save old header in tsv file for lookup.

from Bio import SeqIO

fastafile = '/home/tobiassonva/data/prok2311/prok2311.fasta' 

seqs = SeqIO.parse(fastafile, 'fasta')
print('Writing')
with open(fastafile+'.no_header', 'w') as out:
    header_dict = {}
    for seq in seqs:
        try:
            header = seq.description.split(' ', 1)
            #print(header)
            header_dict[header[0]] = header[1]
        except IndexError:
            header_dict[seq.id] = 'no_header'
        
        seq.description = '' 
        seq.id = header[0]
        SeqIO.write(seq, out, 'fasta')
print('Done')

with open(fastafile+'.header_mapping', 'w') as out:
    for key, value in header_dict.items():
        out.write(f'{key}\t{value}\n')
        

Writing
Done


In [ ]:
#brute force filter taxa assignment by trying several possible taxonomic rank labels
classes = {}
for index, row in eukprot_species_taxa.iterrows():
    lineage = ete3.NCBITaxa().get_lineage(index)
    ranks = ete3.NCBITaxa().get_rank(lineage)
    tax_class = -1
    
    for key, item in ranks.items():
        if item == 'class':
            tax_class = key
            
    if tax_class == -1: 
        #print(index, 'no class')
        for key, item in ranks.items():
            if item == 'phylum':
                tax_class = key

    
    classes[index] = tax_class
    if tax_class == -1 :
        #print(index, 'no phylum')
        for key, item in ranks.items():
            if item == 'superphylum':
                tax_class = key

                
    if tax_class == -1: 
        #print(index, 'no superphylum')
        for key, item in ranks.items():
            if item == 'subkingdom':
                tax_class = key

                
    if tax_class == -1: 
        #print(index, 'no subkingdom')
        for key, item in ranks.items():
            if item == 'kindom':
                tax_class = key
                
    if tax_class == -1: 
        #print(index, 'no kingdom')
        for key, item in ranks.items():
            if item == 'family':
                tax_class = key

    if tax_class == -1: 
        #print(index, 'no family')
        for key, item in ranks.items():
            if item == 'clade':
                tax_class = key
    
    if tax_class == -1:
        print('No taxonomy for ', index)
    
    classes[index] = tax_class
    #print(index, row.species, tax_class)
    
#final stragllers for eukprot
# classes[1246795] = 2686027
# classes[652834] = 759891
# classes[232287] = 589438
# classes[1764928] = 589438
# classes[1155914] = 589438

In [4]:
#remove bad headers from eukprot, slow but only need doing once
from Bio import SeqIO
import numpy as np

cleaned_tsv = '/home/tobiassonva/data/eukgen/processing/search/clean_eukprot/clean_eukprot3.tsv'
fastafile = '/home/tobiassonva/data/eukgen/core_data/euk72_ep/euk72_ep.fasta' 

badDB = pd.read_csv(cleaned_tsv, sep='\t', header=None)
bad_accs = badDB[1].unique()

seqs = SeqIO.parse(fastafile, 'fasta')

print('Writing')
with open(fastafile+'.cleaned', 'w') as out:
    bads = 0
    for seq in seqs:
            if seq.id not in bad_accs:
                SeqIO.write(seq, out, 'fasta')
            else:
                bads += 1
                bad_id = np.argwhere(bad_accs == seq.id)
                bad_accs = np.delete(bad_accs, bad_id)
                print(f'found {bads} bad {seq.id}, {bad_accs.shape}')
print('Done')

Writing
found 1 bad NP_904116.1, (1588,)
found 2 bad XP_006851976.1, (1587,)
found 3 bad NP_051067.1, (1586,)
found 4 bad XP_004342489.2, (1585,)
found 5 bad BAK41516.1, (1584,)
found 6 bad AKU71004.1, (1583,)
found 7 bad ANF99695.1, (1582,)
found 8 bad YP_009423891.1, (1581,)
found 9 bad NP_848982.1, (1580,)
found 10 bad YP_538791.1, (1579,)
found 11 bad NP_001238688.2, (1578,)
found 12 bad XP_012474727.1, (1577,)
found 13 bad ACB87448.1, (1576,)
found 14 bad AOM66863.1, (1575,)
found 15 bad YP_009294626.1, (1574,)
found 16 bad P06267.1, (1573,)
found 17 bad AGK91274.1, (1572,)
found 18 bad APA37186.1, (1571,)
found 19 bad YP_009479646.1, (1570,)
found 20 bad AWF79141.1, (1569,)
found 21 bad AXJ93214.1, (1568,)
found 22 bad XP_002678275.1, (1567,)
found 23 bad XP_001618661.2, (1566,)
found 24 bad XP_032224482.1, (1565,)
found 25 bad YP_009305290.1, (1564,)
found 26 bad NP_039383.1, (1563,)
found 27 bad NP_039416.1, (1562,)
found 28 bad YP_009235020.1, (1561,)
found 29 bad XP_024374492

found 237 bad AZU95207.1, (1352,)
found 238 bad AZU95239.1, (1351,)
found 239 bad APA37190.1, (1350,)
found 240 bad YP_009479650.1, (1349,)
found 241 bad AWF79145.1, (1348,)
found 242 bad XP_001621415.1, (1347,)
found 243 bad XP_032243091.1, (1346,)
found 244 bad XP_001618975.2, (1345,)
found 245 bad XP_032239434.1, (1344,)
found 246 bad YP_052767.1, (1343,)
found 247 bad XP_015610626.1, (1342,)
found 248 bad XP_026422374.1, (1341,)
found 249 bad YP_009477525.1, (1340,)
found 250 bad XP_008915244.1, (1339,)
found 251 bad NP_595257.1, (1338,)
found 252 bad XP_002294514.1, (1337,)
found 253 bad XP_019079136.1, (1336,)
found 254 bad XP_002788075.1, (1335,)
found 255 bad AEY77039.1, (1334,)
found 256 bad YP_004347760.1, (1333,)
found 257 bad YP_009051017.1, (1332,)
found 258 bad XP_006852378.1, (1331,)
found 259 bad NP_051037.1, (1330,)
found 260 bad XP_681133.1, (1329,)
found 261 bad XP_006683154.1, (1328,)
found 262 bad XP_006683282.1, (1327,)
found 263 bad XP_006683346.1, (1326,)
found 

found 472 bad AEY77043.1, (1117,)
found 473 bad AEZ01503.1, (1116,)
found 474 bad AFC39955.1, (1115,)
found 475 bad AFC40019.1, (1114,)
found 476 bad ASN78733.1, (1113,)
found 477 bad ASN78797.1, (1112,)
found 478 bad ASN78829.1, (1111,)
found 479 bad ASN78861.1, (1110,)
found 480 bad YP_009413225.1, (1109,)
found 481 bad YP_009413385.1, (1108,)
found 482 bad CEM37393.1, (1107,)
found 483 bad YP_004347796.1, (1106,)
found 484 bad OLP76110.1, (1105,)
found 485 bad XP_005765446.1, (1104,)
found 486 bad NP_904078.1, (1103,)
found 487 bad NP_051076.1, (1102,)
found 488 bad XP_006683318.1, (1101,)
found 489 bad XP_006683350.1, (1100,)
found 490 bad XP_006683382.1, (1099,)
found 491 bad XP_006683446.1, (1098,)
found 492 bad BAG16016.1, (1097,)
found 493 bad BAG16048.1, (1096,)
found 494 bad BAG16080.1, (1095,)
found 495 bad BAG16112.1, (1094,)
found 496 bad BAG16144.1, (1093,)
found 497 bad BAK08831.1, (1092,)
found 498 bad AKU70981.1, (1091,)
found 499 bad RWR98233.1, (1090,)
found 500 bad 

found 707 bad YP_004347801.1, (882,)
found 708 bad YP_277313.1, (881,)
found 709 bad YP_277345.1, (880,)
found 710 bad XP_005772560.1, (879,)
found 711 bad XP_006683291.1, (878,)
found 712 bad XP_006683323.1, (877,)
found 713 bad XP_006683483.1, (876,)
found 714 bad BAK41530.1, (875,)
found 715 bad GBG69727.1, (874,)
found 716 bad AKU70954.1, (873,)
found 717 bad RWR81726.1, (872,)
found 718 bad YP_009423936.1, (871,)
found 719 bad NP_849094.1, (870,)
found 720 bad YP_003289252.1, (869,)
found 721 bad XP_006577323.1, (868,)
found 722 bad YP_005087730.1, (867,)
found 723 bad XP_012459367.1, (866,)
found 724 bad ACB87462.1, (865,)
found 725 bad YP_009294736.1, (864,)
found 726 bad PTQ34241.1, (863,)
found 727 bad YP_009646850.1, (862,)
found 728 bad AHX03048.1, (861,)
found 729 bad BBD75085.1, (860,)
found 730 bad AXJ93230.1, (859,)
found 731 bad AXJ93264.1, (858,)
found 732 bad QBE89571.1, (857,)
found 733 bad XP_001618031.2, (856,)
found 734 bad XP_001618624.2, (855,)
found 735 bad XP_

found 941 bad XP_006838360.1, (648,)
found 942 bad XP_006683328.1, (647,)
found 943 bad XP_006683392.1, (646,)
found 944 bad XP_006683424.1, (645,)
found 945 bad XP_006683552.1, (644,)
found 946 bad BAG16026.1, (643,)
found 947 bad BAG16058.1, (642,)
found 948 bad BAG16090.1, (641,)
found 949 bad BAG16122.1, (640,)
found 950 bad BAG16154.1, (639,)
found 951 bad AKU70991.1, (638,)
found 952 bad RWR97763.1, (637,)
found 953 bad YP_009423911.1, (636,)
found 954 bad YP_009423941.1, (635,)
found 955 bad CAV31212.1, (634,)
found 956 bad YP_003289257.1, (633,)
found 957 bad YP_003289289.1, (632,)
found 958 bad YP_538789.1, (631,)
found 959 bad XP_012467526.1, (630,)
found 960 bad AOM66882.1, (629,)
found 961 bad AOM67010.1, (628,)
found 962 bad AOM67042.1, (627,)
found 963 bad YP_009294773.1, (626,)
found 964 bad P06251.3, (625,)
found 965 bad P06368.1, (624,)
found 966 bad APA37194.1, (623,)
found 967 bad YP_009479634.1, (622,)
found 968 bad AXJ93269.1, (621,)
found 969 bad QBE89608.1, (620,

found 1171 bad XP_001324767.1, (418,)
found 1172 bad YP_567108.1, (417,)
found 1173 bad NP_043009.1, (416,)
found 1174 bad XP_011409846.1, (415,)
found 1175 bad XP_002778525.1, (414,)
found 1176 bad XP_007394458.1, (413,)
found 1177 bad ASN78812.1, (412,)
found 1178 bad YP_009413272.1, (411,)
found 1179 bad YP_009413336.1, (410,)
found 1180 bad YP_009413368.1, (409,)
found 1181 bad YP_009413400.1, (408,)
found 1182 bad YP_004347747.1, (407,)
found 1183 bad YP_004347811.1, (406,)
found 1184 bad YP_009051103.1, (405,)
found 1185 bad YP_277355.1, (404,)
found 1186 bad NP_051058.1, (403,)
found 1187 bad NP_179709.1, (402,)
found 1188 bad XP_006676494.1, (401,)
found 1189 bad XP_006683269.1, (400,)
found 1190 bad XP_006683365.1, (399,)
found 1191 bad BAK41540.1, (398,)
found 1192 bad NP_849105.1, (397,)
found 1193 bad CBH31935.1, (396,)
found 1194 bad CAT18727.1, (395,)
found 1195 bad YP_538781.1, (394,)
found 1196 bad XP_003535239.1, (393,)
found 1197 bad AAZ76070.1, (392,)
found 1198 bad 

found 1402 bad XP_012487294.1, (187,)
found 1403 bad XP_012464985.1, (186,)
found 1404 bad AEK70380.1, (185,)
found 1405 bad AJQ31802.1, (184,)
found 1406 bad YP_009294622.1, (183,)
found 1407 bad YP_009294750.1, (182,)
found 1408 bad YP_009294782.1, (181,)
found 1409 bad GBG26949.1, (180,)
found 1410 bad P06248.1, (179,)
found 1411 bad P26856.2, (178,)
found 1412 bad AZU95199.1, (177,)
found 1413 bad YP_009646820.1, (176,)
found 1414 bad YP_009646846.1, (175,)
found 1415 bad APA37182.1, (174,)
found 1416 bad BBD75067.1, (173,)
found 1417 bad BBD75131.1, (172,)
found 1418 bad YP_009479642.1, (171,)
found 1419 bad AXJ93210.1, (170,)
found 1420 bad AXJ93245.1, (169,)
found 1421 bad OAD05208.1, (168,)
found 1422 bad XP_002674303.1, (167,)
found 1423 bad XP_001617889.2, (166,)
found 1424 bad XP_001621425.2, (165,)
found 1425 bad XP_031502390.1, (164,)
found 1426 bad YP_654226.2, (163,)
found 1427 bad NP_039445.1, (162,)
found 1428 bad YP_009235016.1, (161,)
found 1429 bad NP_904226.1, (160

In [ ]:
eukprot_species_taxa['EukProtID'] = [eukprot_data[eukprot_data.taxid == tax].index.values[0] for tax in eukprot_species_taxa.index]
eukprot_species_taxa.columns = ['species', 'superkingdom', 'class', 'EukProtID']
eukprot_species_taxa

In [ ]:
with open('core_data/taxonomy/eukprot.data.pkl', 'wb') as out:
    pickle.dump(eukprot_data, out)
    
eukprot_data.to_csv('core_data/taxonomy/eukprot.data.tsv', sep='\t')

In [ ]:
with open('core_data/eukprot_species_taxa.pkl', 'rb') as infile:
    eukprot_species_taxa = pickle.load(infile)

In [ ]:
eukprot_species_taxa

In [ ]:
set(classes.values()).difference(class_translator.values())

In [ ]:
eukprot_species_taxa

In [ ]:
pd.Series(ete3.NCBITaxa().get_rank(list(classes.values()))).value_counts()

In [ ]:
len(classes.values())

In [ ]:
bad = [key for key, value in classes.items() if value==-1]
print(bad)

In [ ]:
# format prok2111_as from merger of prok211 and ettema2023 datasets
# use preexisting prok taxonomy and import manuslly curated taxids for ettema2023

#load prok header info
with open('core_data/taxonomy/prok2111_protein_taxonomy.pkl', 'rb') as infile:
    prot_data = pickle.load(infile)

#read manual curated asgard header files
asgard_seqs = pd.read_csv('core_data/ettema2023/asgard_proteomes2023_header_mapping.tsv', sep ='\t', names=['acc','header'])
asgard_acc_species_map = pd.read_csv('core_data/ettema2023/asgard_acc_species_mapping.tsv', sep='\t', names=['acc', 'species', 'orgid', 'class'], index_col=0)

#format protein_taxonomy table
asgard_seqs[['orgid', 'species', 'superkingdom', 'class']] = [[asgard_acc_species_map.loc[acc.split('..')[0]]['orgid'], asgard_acc_species_map.loc[acc.split('..')[0]].species, 'Archaea', 'Asgard'] for acc in asgard_seqs.acc]
asgard_seqs.set_index('acc', inplace=True)
asgard_seqs.drop('header', axis=1, inplace=True)

#write to tsv and pickle
asgard_seqs.to_csv('core_data/ettema2023/asgard2023_protein_taxonomy.tsv', sep='\t')
with open('core_data/ettema2023/asgard2023_protein_taxonomy.pkl', 'wb') as pkl:
    pickle.dump(asgard_seqs, pkl)


In [ ]:
#additional files

merged = pd.concat([prot_data, asgard_seqs])
merged.to_csv('core_data/prok2111_as/prok2111_as_protein_taxonomy.tsv', sep='\t', header=None)
taxdb_cols = merged[['orgid']]
taxdb_cols.to_csv('core_data/prok2111_as/prok2111.tax', header=None, sep='\t')

In [ ]:
with open('core_data/ettema2023/asgard2023_protein_taxonomy.pkl', 'rb') as pkl:
    asgard_protein_tax = pickle.load(pkl)

In [ ]:
prot_data

In [ ]:
with open('core_data/taxonomy/euk72_protein_taxonomy.pkl', 'rb') as infile:
    test = pickle.load(infile)
test.drop_duplicates('orgid').set_index('orgid', drop=True)

In [ ]:
with open('core_data/euk72_species_taxa.pkl', 'wb') as out:
    pickle.dump(test.drop_duplicates('orgid').set_index('orgid', drop=True), out)
    
with open('core_data/euk72_species_taxa.pkl', 'rb') as infile:
    test = pickle.load(infile)
    
test

In [ ]:
asgard_protein_tax.drop_duplicates('orgid')

In [2]:
#format cdd
import os
from Bio import SeqIO

%cd ~/data/databases/cdd/

/vf/users/tobiassonva/data/databases/cdd


In [5]:
header = pd.read_csv('cddid_parsed.tsv', sep='\t')
header

for f in os.listdir('cdd_fastas/'):
    seqs = [*SeqIO.parse('cdd_fastas/'+f, 'fasta')]
    seqs[0].id = f
    seqs[0].description = ''
    
    print(seqs[0].id)
    with open(f'cdd_fastas_renamed/{f}', 'w') as out:
        SeqIO.write(seqs, out, 'fasta')

CHL00001
CHL00002
CHL00003
CHL00004
CHL00005
CHL00008
CHL00009
CHL00010
CHL00011
CHL00012
CHL00013
CHL00014
CHL00015
CHL00016
CHL00017
CHL00018
CHL00019
CHL00020
CHL00022
CHL00023
CHL00024
CHL00025
CHL00027
CHL00028
CHL00029
CHL00030
CHL00031
CHL00032
CHL00033
CHL00034
CHL00035
CHL00036
CHL00037
CHL00038
CHL00039
CHL00040
CHL00041
CHL00042
CHL00043
CHL00044
CHL00045
CHL00046
CHL00047
CHL00048
CHL00049
CHL00050
CHL00051
CHL00052
CHL00053
CHL00054
CHL00056
CHL00057
CHL00058
CHL00059
CHL00060
CHL00061
CHL00062
CHL00063
CHL00064
CHL00065
CHL00066
CHL00067
CHL00068
CHL00070
CHL00071
CHL00072
CHL00073
CHL00074
CHL00075
CHL00076
CHL00077
CHL00078
CHL00079
CHL00080
CHL00081
CHL00082
CHL00083
CHL00084
CHL00085
CHL00086
CHL00088
CHL00089
CHL00090
CHL00091
CHL00093
CHL00094
CHL00095
CHL00098
CHL00099
CHL00100
CHL00101
CHL00102
CHL00103
CHL00104
CHL00105
CHL00106
CHL00108
CHL00112
CHL00113
CHL00114
CHL00115
CHL00117
CHL00118
CHL00119
CHL00120
CHL00121
CHL00122
CHL00123
CHL00124
CHL00125
CHL00127
C

COG0863
COG0864
COG1001
COG1002
COG1003
COG1004
COG1005
COG1006
COG1007
COG1008
COG1009
COG1010
COG1011
COG1012
COG1013
COG1014
COG1015
COG1017
COG1018
COG1019
COG1020
COG1021
COG1022
COG1023
COG1024
COG1025
COG1026
COG1027
COG1028
COG1029
COG1030
COG1031
COG1032
COG1033
COG1034
COG1035
COG1036
COG1038
COG1039
COG1040
COG1041
COG1042
COG1043
COG1044
COG1045
COG1047
COG1048
COG1049
COG1051
COG1052
COG1053
COG1054
COG1055
COG1056
COG1057
COG1058
COG1059
COG1060
COG1061
COG1062
COG1063
COG1064
COG1066
COG1067
COG1069
COG1070
COG1071
COG1072
COG1073
COG1074
COG1075
COG1076
COG1077
COG1078
COG1079
COG1080
COG1082
COG1083
COG1084
COG1085
COG1086
COG1087
COG1088
COG1089
COG1090
COG1091
COG1092
COG1093
COG1094
COG1095
COG1096
COG1097
COG1098
COG1099
COG1100
COG1101
COG1102
COG1103
COG1104
COG1105
COG1106
COG1107
COG1108
COG1109
COG1110
COG1111
COG1112
COG1113
COG1114
COG1115
COG1116
COG1117
COG1118
COG1119
COG1120
COG1121
COG1122
COG1123
COG1124
COG1125
COG1126
COG1127
COG1129
COG1131
COG1132


COG2082
COG2083
COG2084
COG2085
COG2086
COG2087
COG2088
COG2089
COG2090
COG2091
COG2092
COG2093
COG2094
COG2095
COG2096
COG2097
COG2098
COG2099
COG2100
COG2101
COG2102
COG2103
COG2104
COG2105
COG2106
COG2107
COG2108
COG2109
COG2110
COG2111
COG2112
COG2113
COG2114
COG2115
COG2116
COG2117
COG2118
COG2119
COG2120
COG2121
COG2122
COG2123
COG2124
COG2125
COG2126
COG2127
COG2128
COG2129
COG2130
COG2131
COG2132
COG2133
COG2134
COG2135
COG2136
COG2137
COG2138
COG2139
COG2140
COG2141
COG2142
COG2143
COG2144
COG2145
COG2146
COG2147
COG2148
COG2149
COG2150
COG2151
COG2152
COG2153
COG2154
COG2155
COG2156
COG2157
COG2158
COG2159
COG2160
COG2161
COG2162
COG2163
COG2164
COG2165
COG2166
COG2167
COG2168
COG2169
COG2170
COG2171
COG2172
COG2173
COG2174
COG2175
COG2176
COG2177
COG2178
COG2179
COG2180
COG2181
COG2182
COG2183
COG2184
COG2185
COG2186
COG2187
COG2188
COG2189
COG2190
COG2191
COG2192
COG2193
COG2194
COG2195
COG2197
COG2198
COG2199
COG2200
COG2201
COG2202
COG2203
COG2204
COG2205
COG2206
COG2207


COG3503
COG3504
COG3505
COG3506
COG3507
COG3508
COG3509
COG3510
COG3511
COG3512
COG3513
COG3514
COG3515
COG3516
COG3517
COG3518
COG3519
COG3520
COG3521
COG3522
COG3523
COG3524
COG3525
COG3526
COG3527
COG3528
COG3529
COG3530
COG3531
COG3533
COG3534
COG3535
COG3536
COG3537
COG3538
COG3539
COG3540
COG3541
COG3542
COG3543
COG3544
COG3545
COG3546
COG3547
COG3548
COG3549
COG3550
COG3551
COG3552
COG3553
COG3554
COG3555
COG3556
COG3557
COG3558
COG3559
COG3560
COG3561
COG3562
COG3563
COG3564
COG3565
COG3566
COG3567
COG3568
COG3569
COG3570
COG3571
COG3572
COG3573
COG3575
COG3576
COG3577
COG3579
COG3580
COG3581
COG3582
COG3583
COG3584
COG3585
COG3586
COG3587
COG3588
COG3589
COG3590
COG3591
COG3592
COG3593
COG3594
COG3595
COG3596
COG3597
COG3598
COG3599
COG3600
COG3601
COG3602
COG3603
COG3604
COG3605
COG3607
COG3608
COG3609
COG3610
COG3611
COG3612
COG3613
COG3614
COG3615
COG3616
COG3617
COG3618
COG3619
COG3620
COG3621
COG3622
COG3623
COG3624
COG3625
COG3626
COG3627
COG3628
COG3629
COG3630
COG3631


COG4731
COG4732
COG4733
COG4734
COG4735
COG4736
COG4737
COG4738
COG4739
COG4740
COG4741
COG4742
COG4743
COG4744
COG4745
COG4746
COG4747
COG4748
COG4749
COG4750
COG4752
COG4753
COG4754
COG4755
COG4756
COG4757
COG4758
COG4759
COG4760
COG4762
COG4763
COG4764
COG4765
COG4766
COG4767
COG4768
COG4769
COG4770
COG4771
COG4772
COG4773
COG4774
COG4775
COG4776
COG4778
COG4779
COG4781
COG4782
COG4783
COG4784
COG4785
COG4786
COG4787
COG4789
COG4790
COG4791
COG4792
COG4794
COG4795
COG4796
COG4797
COG4798
COG4799
COG4800
COG4801
COG4802
COG4803
COG4804
COG4805
COG4806
COG4807
COG4808
COG4809
COG4810
COG4811
COG4812
COG4813
COG4814
COG4815
COG4816
COG4817
COG4818
COG4819
COG4820
COG4821
COG4822
COG4823
COG4824
COG4825
COG4826
COG4827
COG4828
COG4829
COG4830
COG4831
COG4832
COG4833
COG4834
COG4835
COG4836
COG4837
COG4838
COG4839
COG4840
COG4841
COG4842
COG4843
COG4844
COG4845
COG4846
COG4847
COG4848
COG4849
COG4850
COG4851
COG4852
COG4853
COG4854
COG4855
COG4856
COG4857
COG4858
COG4859
COG4860
COG4861


...

pfam16897
pfam16898
pfam16899
pfam16900
pfam16901
pfam16902
pfam16903
pfam16904
pfam16905
pfam16906
pfam16907
pfam16908
pfam16909
pfam16910
pfam16911
pfam16912
pfam16913
pfam16914
pfam16915
pfam16916
pfam16917
pfam16918
pfam16919
pfam16920
pfam16921
pfam16922
pfam16923
pfam16924
pfam16925
pfam16926
pfam16927
pfam16928
pfam16929
pfam16930
pfam16931
pfam16932
pfam16933
pfam16934
pfam16935
pfam16936
pfam16937
pfam16938
pfam16939
pfam16940
pfam16941
pfam16942
pfam16943
pfam16944
pfam16945
pfam16946
pfam16947
pfam16949
pfam16951
pfam16952
pfam16953
pfam16954
pfam16955
pfam16956
pfam16957
pfam16958
pfam16959
pfam16960
pfam16961
pfam16962
pfam16963
pfam16964
pfam16965
pfam16966
pfam16967
pfam16968
pfam16969
pfam16970
pfam16971
pfam16972
pfam16973
pfam16974
pfam16975
pfam16976
pfam16977
pfam16978
pfam16979
pfam16980
pfam16981
pfam16982
pfam16983
pfam16984
pfam16985
pfam16986
pfam16987
pfam16988
pfam16989
pfam16990
pfam16991
pfam16992
pfam16993
pfam16994
pfam16995
pfam16996
pfam16997
pfam16998


pfam17761
pfam17762
pfam17763
pfam17764
pfam17765
pfam17766
pfam17767
pfam17768
pfam17769
pfam17770
pfam17771
pfam17772
pfam17773
pfam17774
pfam17775
pfam17776
pfam17777
pfam17778
pfam17779
pfam17780
pfam17781
pfam17782
pfam17783
pfam17784
pfam17785
pfam17786
pfam17787
pfam17788
pfam17789
pfam17790
pfam17791
pfam17792
pfam17793
pfam17794
pfam17795
pfam17796
pfam17797
pfam17798
pfam17799
pfam17800
pfam17801
pfam17802
pfam17803
pfam17804
pfam17805
pfam17806
pfam17807
pfam17808
pfam17809
pfam17810
pfam17811
pfam17812
pfam17813
pfam17814
pfam17815
pfam17816
pfam17817
pfam17818
pfam17819
pfam17820
pfam17821
pfam17822
pfam17823
pfam17824
pfam17825
pfam17826
pfam17827
pfam17828
pfam17829
pfam17830
pfam17831
pfam17832
pfam17833
pfam17834
pfam17835
pfam17836
pfam17837
pfam17838
pfam17839
pfam17840
pfam17841
pfam17842
pfam17843
pfam17844
pfam17845
pfam17846
pfam17847
pfam17848
pfam17849
pfam17850
pfam17851
pfam17852
pfam17853
pfam17854
pfam17855
pfam17856
pfam17857
pfam17858
pfam17859
pfam17860


pfam18599
pfam18600
pfam18601
pfam18602
pfam18603
pfam18604
pfam18606
pfam18607
pfam18608
pfam18609
pfam18610
pfam18611
pfam18612
pfam18613
pfam18614
pfam18615
pfam18616
pfam18617
pfam18618
pfam18619
pfam18620
pfam18621
pfam18622
pfam18623
pfam18624
pfam18625
pfam18626
pfam18627
pfam18628
pfam18629
pfam18630
pfam18631
pfam18632
pfam18633
pfam18634
pfam18635
pfam18636
pfam18637
pfam18638
pfam18639
pfam18640
pfam18641
pfam18642
pfam18643
pfam18644
pfam18645
pfam18646
pfam18647
pfam18648
pfam18649
pfam18650
pfam18651
pfam18652
pfam18653
pfam18654
pfam18655
pfam18656
pfam18657
pfam18658
pfam18659
pfam18660
pfam18661
pfam18662
pfam18663
pfam18664
pfam18665
pfam18666
pfam18667
pfam18668
pfam18669
pfam18670
pfam18671
pfam18672
pfam18673
pfam18674
pfam18675
pfam18676
pfam18677
pfam18678
pfam18679
pfam18680
pfam18681
pfam18682
pfam18683
pfam18684
pfam18685
pfam18686
pfam18687
pfam18688
pfam18689
pfam18690
pfam18691
pfam18692
pfam18693
pfam18694
pfam18695
pfam18696
pfam18697
pfam18698
pfam18699


pfam19440
pfam19441
pfam19442
pfam19443
pfam19444
pfam19445
pfam19446
pfam19447
pfam19448
pfam19449
pfam19450
pfam19451
pfam19452
pfam19453
pfam19454
pfam19455
pfam19456
pfam19457
pfam19458
pfam19459
pfam19460
pfam19461
pfam19462
pfam19463
pfam19464
pfam19465
pfam19466
pfam19467
pfam19468
pfam19469
pfam19470
pfam19471
pfam19472
pfam19473
pfam19474
pfam19475
pfam19476
pfam19477
pfam19478
pfam19479
pfam19480
pfam19481
pfam19482
pfam19483
pfam19484
pfam19485
pfam19486
pfam19487
pfam19488
pfam19489
pfam19490
pfam19491
pfam19492
pfam19493
pfam19494
pfam19495
pfam19496
pfam19497
pfam19498
pfam19499
pfam19500
pfam19501
pfam19502
pfam19503
pfam19504
pfam19505
pfam19506
pfam19507
pfam19508
pfam19509
pfam19510
pfam19511
pfam19512
pfam19513
pfam19514
pfam19515
pfam19516
pfam19517
pfam19518
pfam19519
pfam19520
pfam19521
pfam19522
pfam19523
pfam19524
pfam19525
pfam19526
pfam19527
pfam19528
pfam19529
pfam19530
pfam19531
pfam19532
pfam19533
pfam19534
pfam19535
pfam19536
pfam19537
pfam19538
pfam19539


IsADirectoryError: [Errno 21] Is a directory: 'cdd_fastas/renamed'

In [ ]:
from Bio import SeqIO

fastafile = './core_data/new_euk72_ep/merged.fasta' 

seqs = SeqIO.parse(fastafile, 'fasta')
print('Writing')
with open(fastafile+'.no_header', 'w') as out:
    header_dict = {}
    for seq in seqs:
        try:
            header = seq.description.split(' ', 1)
            #print(header)
            header_dict[header[0]] = header[1]
        except IndexError:
            header_dict[seq.id] = 'no_header'
        
        seq.description = '' 
        seq.id = header[0]
        SeqIO.write(seq, out, 'fasta')
print('Done')

with open(fastafile+'.header_mapping', 'w') as out:
    for key, value in header_dict.items():
        out.write(f'{key}\t{value}\n')